In [1]:
%matplotlib inline
%load_ext autoreload

import matplotlib.pyplot as plt
import numpy as np
import cPickle as pickle
import sys
import os
import copy

sys.path.append('/Users/Michael/projects/shape_sharing/src/')
from common import paths
from common import parameters

In [2]:
with open('/tmp/partial_segmented.pkl', 'r') as f:
    grids = pickle.load(f)

In [3]:
partial_tsdf = grids['partial_tsdf']
im = grids['image']

partial_tsdf_nonan = copy.deepcopy(partial_tsdf)
partial_tsdf_nonan.V[
    np.isnan(partial_tsdf_nonan.V)] = np.nanmin(partial_tsdf_nonan.V)

plt.subplot(121)
plt.imshow(partial_tsdf_nonan.V[:, :, 15])
plt.subplot(122)
plt.imshow(im.depth);

In [5]:
# def project_unobserved_voxels(im, grid):
#     # project the nan voxels from grid into the image...
#     to_project_idxs = np.where(grid.V.flatten() != np.nanmax(grid.V))[0]
#     nan_xyz = grid.world_meshgrid()[to_project_idxs]
#     return im.cam.project_points(nan_xyz).astype(np.int32), to_project_idxs

uv, to_project_idxs = partial_tsdf.project_unobserved_voxels(im)
inside_image = im.find_points_inside_image(uv)

# labels of all the non-nan voxels inside the image...
vox_labels = im.labels[uv[inside_image, 1], uv[inside_image, 0]]
print "Labels present in image are ", np.unique(vox_labels)

# now propograte these labels back to the main grid
temp = to_project_idxs[inside_image]
partial_tsdf.labels.ravel()[temp] = vox_labels

plt.imshow(partial_tsdf.labels[:, :, 25]);

In [20]:
def separate_binary_grids(vox_labels, flatten_labels_down):
    '''
    takes a 3D array of labels, and returns a dict
    where keys are labels and values are a 3D boolean
    array with one where that label is true
    If flatten_labels_down == True, then cumsums along
    the -ve z axis.
    '''    
    each_label_region = {}

    for idx in np.unique(vox_labels):
        label_temp = partial_tsdf.labels == idx

        if flatten_labels_down:
            label_temp = np.cumsum(
                label_temp[:, :, ::-1], axis=2)[:, :, ::-1]

        each_label_region[idx] = label=label_temp > 0
        
    return each_label_region


def label_grids_to_tsdf_grids(tsdf, binary_grids):
    '''
    extracts the tsdf corresponding to each of the binary grids
    '''
    tsdf_grids = {}
    for idx, reg in binary_grids.iteritems():
        temp = tsdf.copy()
        temp.V[~reg] = np.nanmax(tsdf.V)
        tsdf_grids[idx] = temp
        
    return tsdf_grids

labels_grids = separate_binary_grids(vox_labels, True)
tsdf_grids = \
    label_grids_to_tsdf_grids(partial_tsdf_nonan, labels_grids)

plt.figure(figsize=(10, 10))
for idx, reg in tsdf_grids.iteritems():

    plt.subplot(3, 3, idx+1)
    plt.imshow(reg.V[:, :, 10])


In [ ]:
# I think I will have to project all the nan voxels into the camera
# and then find out which voxels lie in the image mask defined
# by this segment... This is quite complex but I think neccesary,
# and it only needs to be done once per segment
